# Metrics on scikit-learn github repo

In [ ]:
import json
import datetime
import numpy as np
import pandas as pd

In [ ]:
with open("../data/pullrequests.json") as f:
    prstr = f.read()

In [ ]:
prjson = json.loads(prstr)
ntot = len(prjson)
ntot

In [ ]:
creatDate = [ prjson[i]["node"]["createdAt"] for i in range(0, ntot) ]
closDate = [ prjson[i]["node"]["closedAt"] for i in range(0, ntot) ]
number = [ prjson[i]["node"]["number"] for i in range(0, ntot) ]
state = [ prjson[i]["node"]["state"] for i in range(0, ntot) ]
lastEd = [ prjson[i]["node"]["lastEditedAt"] for i in range(0, ntot) ]
nreview = [ prjson[i]["node"]["reviews"]["totalCount"] for i in range(0, ntot) ]

In [ ]:
author = []
firstcommentedBy = []
firstcommentedAt = []
for i in range(0, ntot):
    auth = prjson[i]["node"]["author"]
    if (auth != None):
        author.append(auth["login"])
    else:
        author.append("")
    comments = prjson[i]["node"]["comments"]["edges"]
    if (len(comments) > 0):
        if (comments[0]["node"]["author"] != None):
            firstcommentedBy.append(comments[0]["node"]["author"]["login"])
        else:
            firstcommentedBy.append("")
        firstcommentedAt.append(comments[0]["node"]["createdAt"])
    else:    
        firstcommentedBy.append("")
        firstcommentedAt.append("")

In [ ]:
from datetime import datetime
from datetime import timedelta

crdate = []
cldate = []
ledate = []
answdate = []
duration = []
reaction = []
delta = timedelta(hours=1)

for i in range(0, ntot):
    crdate.append(pd.to_datetime(creatDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    cldate.append(pd.to_datetime(closDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    ledate.append(pd.to_datetime(lastEd[i], format='%Y-%m-%dT%H:%M:%SZ'))
    if closDate[i] != None:
        duration.append(pd.Timedelta.to_pytimedelta(cldate[i] - crdate[i]) / delta )
    else:
        duration.append(None)
    try:
        answdate.append(pd.to_datetime(firstcommentedAt[i], format='%Y-%m-%dT%H:%M:%SZ'))
        reaction.append(pd.Timedelta.to_pytimedelta(answdate[i] - crdate[i]) / delta )
    except:
        answdate.append(None)
        reaction.append(None)

In [ ]:
ntot

In [ ]:
prdata = { "Number" : number, "CreatedAt" : crdate, "ClosedAt" : cldate,
           "Duration" : duration, "lastEditedAt" : ledate, "Author" : author,
           "State" : state, "nReviews" : nreview, "FcommAuthor" : firstcommentedBy,
           "FcommDate" : answdate, "AnswerTime" : reaction }
prdf = pd.DataFrame(prdata)

In [ ]:
prdf.head()

In [ ]:
import pandas_profiling
prdf.profile_report()

In [ ]:
gprdf = prdf.groupby([prdf['CreatedAt'].dt.to_period('Y')]).count().unstack()
gprdf['CreatedAt'].plot(kind='bar', color='red')
gprdf['ClosedAt'].plot(kind='bar', color='blue')